# LAS Data Dive Group F

In [25]:
import numpy as np
import pandas as pd
import geopandas as gpd
import re
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Read in data

In [26]:
LEAVERS = pd.read_csv("data/Workforce Data/LAS_Leavers_070319.csv")
STAFF = pd.read_csv("data/Workforce Data/LAS_Staff_in_Post_070319.csv")
STATIONS = pd.read_csv("data/Workforce Data/workplace location table.csv")


#From https://datashare.is.ed.ac.uk/handle/10283/2597
PC_SECTOR = gpd.read_file("data/postcode/Sectors.shp", crs={'init': 'epsg:4326'})

In [27]:
STAFF["LengthOfService(Years)"] = pd.to_numeric(STAFF["LengthOfService(Years)"], float)

STAFF["LengthOfService(Months)"] = pd.to_numeric(STAFF["LengthOfService(Months)"], float)

STAFF["LengthOfServiceFLOAT"] = STAFF["LengthOfService(Years)"] + STAFF["LengthOfService(Months)"]/12

## Adding geospatial info to current staff table

In [28]:
#Add station postcodes to staff data
STAFFM2 = pd.merge(STAFF, STATIONS, left_on="Location", right_on="esrLocationFull")
STAFFM2 = STAFFM2.rename(columns={"Location" : "WorkLocation", "postcode" : "PostcodeWork"})
STAFFM2

,IDnumber,Board,Division,Directorate,Locality,Service,Department,WorkLocation,Gender,AgeBand,PostcodeSector,PositionTitle,EmployeeCategory,AssignmentCategory,FTE,ContractHours,Frequency,LengthOfService(Years),LengthOfService(Months),nhsEntryDate,LatestStartDate,StartDateInGrade,TimeInGrade(Years),AssignmentEffectiveStartDate,OriginalHireDate,StartDateInPosition,LengthOfServiceFLOAT,location,locationaddress,PostcodeWork,esrLocationFull,manual
0,25408862,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Male,26-30,EN3 7,Emergency Ambulance Crew,Full Time,Permanent,1.00,37.5,Week,3,5,14/09/2015,14/09/2015,26/06/2017,1.70,18/01/2019,14/09/2015,18/01/2019,3.416667,Edmonton,Edmonton Ambulance Station Windmill Road Londo...,N18 1NJ,308 28 H3 Edmonton,0
1,10547942,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Male,51-55,CM4 0,Paramedic B6,Full Time,Permanent,1.00,37.5,Week,14,4,25/10/2004,25/10/2004,31/12/2016,2.18,21/02/2019,25/10/2004,31/12/2016,14.333333,Edmonton,Edmonton Ambulance Station Windmill Road Londo...,N18 1NJ,308 28 H3 Edmonton,0
2,24119591,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Male,26-30,EN9 3,Paramedic B6,Full Time,Permanent,1.00,37.5,Week,4,4,14/10/2013,01/11/2014,31/12/2016,2.18,31/12/2018,14/10/2013,24/07/2018,4.333333,Edmonton,Edmonton Ambulance Station Windmill Road Londo...,N18 1NJ,308 28 H3 Edmonton,0
3,23892265,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Male,26-30,E1 5,Emergency Ambulance Crew,Full Time,Permanent,1.00,37.5,Week,5,8,10/06/2013,10/06/2013,25/07/2016,2.62,27/12/2018,10/06/2013,04/12/2018,5.666667,Edmonton,Edmonton Ambulance Station Windmill Road Londo...,N18 1NJ,308 28 H3 Edmonton,0
4,20213785,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Male,46-50,AL1 3,Emergency Ambulance Crew,Full Time,Permanent,1.00,37.5,Week,11,11,04/04/2005,26/03/2007,21/07/2014,4.63,21/07/2018,26/03/2007,27/01/2017,11.916667,Edmonton,Edmonton Ambulance Station Windmill Road Londo...,N18 1NJ,308 28 H3 Edmonton,0
5,27334005,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Female,21-25,N13 4,Paramedic NQ,Full Time,Permanent,1.00,37.5,Week,0,8,11/06/2018,11/06/2018,11/06/2018,0.74,11/06/2018,11/06/2018,11/06/2018,0.666667,Edmonton,Edmonton Ambulance Station Windmill Road Londo...,N18 1NJ,308 28 H3 Edmonton,0
6,10547356,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Female,31-35,SG18 8,Paramedic B6,Full Time,Permanent,1.00,37.5,Week,11,7,12/09/2006,30/07/2007,31/12/2016,2.18,30/07/2018,01/11/2004,31/12/2016,11.583333,Edmonton,Edmonton Ambulance Station Windmill Road Londo...,N18 1NJ,308 28 H3 Edmonton,0
7,20722214,308 Operations L3,308 OPS Operations Directorate L4,308 SEC Sector Operations L5,308 SEC North Central Sector L6,308 SEC Edmonton Group Station L7,308 SEC Edmonton Ambulance Station,308 28 H3 Edmonton,Male,46-50,EN10 6,Paramedic B6,Part Time,Permanent,0.61,23.0,Week,10,11,05/03/2008,07/04/2008,31/12/2016,2.18,31/12/2018,07/04/2008,31/12/2016,10.916667

In [29]:
#Function to extract coordinates
def extract_coords(point):
    lat = re.findall(r'[-]*[0-9]+[.]*[0-9]+', str(point))
    long = re.findall(r'\s([0-9]+[.]*[0-9]+)', str(point))
    return str(long[0] + "," + lat[0])

In [30]:
#Merge staff data with postcode sector boundaries
STAFF_PC_AGG_GEO = pd.merge(STAFF, PC_SECTOR, left_on="PostcodeSector", right_on="name")

#Create geodatamframe
STAFF_PC_AGG_GEO = gpd.GeoDataFrame(STAFF_PC_AGG_GEO, geometry='geometry', crs={'init': 'epsg:4326'})

#Get centroids
STAFF_PC_AGG_GEO["centroids"] = STAFF_PC_AGG_GEO.centroid

STAFF_PC_AGG_GEO["centroids_str"] = STAFF_PC_AGG_GEO["centroids"].apply(extract_coords)

In [31]:
#Get postcodes of departments
INFERR = pd.DataFrame(STAFFM2.groupby(by=["Department", "WorkLocation", "PostcodeWork"]).count()["IDnumber"].sort_values(ascending = False))
INFERR.columns = ["Count"]

INFERR = INFERR.reset_index()

INFERR2 = INFERR.set_index(["Department", "WorkLocation"])

#Input this in to online postcode search website https://gridreferencefinder.com/postcodeBatchConverter/
INFERR2.to_csv("Org Department postcodes.csv")

In [32]:
#Import this data
Workplace_coords = pd.read_csv("station_pc_latlong.csv")
Workplace_coords

,Postcode,Latitude,Longitude,"Lat,long"
0,DA7 6BZ,51.460793,0.153806,"51.460793,0.15380635"
1,EN5 1TE,51.647271,-0.185914,"51.647271,-0.18591394"
2,SW11 1HW,51.460613,-0.167788,"51.460613,-0.16778757"
3,BR3 4LR,51.408006,-0.034010,"51.408006,-0.034009912"
4,WC1N 1HP,51.524889,-0.126632,"51.524889,-0.12663222"
5,N22 6AD,51.592380,-0.096643,"51.59238,-0.096642725"
6,E3 3PA,51.521231,-0.015227,"51.521231,-0.015227053"
7,E3 3PA,51.521231,-0.015227,"51.521231,-0.015227053"
8,NW10 1RZ,51.555041,-0.248931,"51.555041,-0.24893072"
9,SE24 0HG,51.458974,-0.102615,"51.458974,-0.10261537"


In [47]:
#Add co-ordinates to staff data
MERGED_MEGA = pd.merge(STAFFM2, Workplace_coords,  left_on="PostcodeWork", right_on="Postcode", how="left")

#Export to csv for use in Tableau
MERGED_MEGA.to_csv("OD COUNTS5.csv")

## Calculating turnover

In [40]:
STAFF["LatestStartDate"] = pd.to_datetime(STAFFM2["LatestStartDate"])
LEAVERS["Employee Latest Start Date"] = pd.to_datetime(LEAVERS["Employee Latest Start Date"])
LEAVERS["Termination Date"] = pd.to_datetime(LEAVERS["Termination Date"])

In [46]:
Leavers_2018 = len(LEAVERS.loc[LEAVERS["Leaving year"] == 2018])

Staff_Dec_2018 = len(STAFF.loc[STAFF["LatestStartDate"] < pd.to_datetime("2018-12-31")])
Staff_Jan_2018 = len(STAFF.loc[STAFF["LatestStartDate"] < pd.to_datetime("2018-01-01")])
Average_staff2018 = (Staff_Dec_2018 + Staff_Jan_2018)/2 + Leavers_2018/2
Average_staff2018

Turnover_Rate = Leavers_2018/Average_staff2018
Turnover_Rate

0.11745028805054822

In [42]:
LEAVERS3 = LEAVERS[["Employee Number", "Employee Latest Start Date", "Termination Date", "Org L5"]]

STAFF["Termination Date"] = np.nan
STAFFM3 = STAFF[["IDnumber", "LatestStartDate", "Directorate", "Termination Date"]]

In [43]:
STAFFM3.columns = ["Employee Number", "Employee Latest Start Date", "Org L5", "Termination Date"]

In [44]:
MERGED_STAFF = pd.concat([LEAVERS3, STAFFM3], axis=0, ignore_index=True)

/Users/student/anaconda3/envs/cusplondon/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [45]:
Leave2018 = MERGED_STAFF.loc[MERGED_STAFF["Termination Date"] < pd.to_datetime("2018-12-31")]
Leave2018 = Leave2018.loc[Leave2018["Termination Date"] > pd.to_datetime("2018-01-01")]